#                           Big Data Programming  - Assignment 2

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

sc = spark.sparkContext

book = sc.textFile("spark-2.4.4-bin-hadoop2.7/README.md")

### STEP1 - Conversation of the given XML files to RDDs

In [3]:
import pprint

import xml.etree.ElementTree as ET
#Users.xml --> UsersRDD
tree = ET.parse('my_new_project/data/Users.xml')
root = tree.getroot()
list1=[]
for child in root:
    list1.append(child.attrib)
UsersRDD = sc.parallelize(list1)

#PostHistory.xml --> PostHistoryRDD
tree = ET.parse('my_new_project/data/PostHistory.xml')
root = tree.getroot()
list1=[]
for child in root:
    list1.append(child.attrib)
PostHistoryRDD = sc.parallelize(list1)

#Comments.xml --> CommentsRDD
tree = ET.parse('my_new_project/data/Comments.xml')
root = tree.getroot()
list1=[]
for child in root:
    list1.append(child.attrib)
CommentsRDD = sc.parallelize(list1)

### Question-1) From the Users.xml file, find all users which are from Georgia and output to screen their DisplayName only. 
### Answer-1) -->

In [5]:

#Step 1 - Filtering out UsersRDD for 'Location' key only, locationLines is the filtered RDD
locationLines= UsersRDD.filter(lambda line: "Location" in line)


#Step 2 - Find out the location Georgia, the cases which can be covered are -
#Case1 -Substring of Georgia || Case2 - Substring of Atlanta || Case3 - Substring of ', GA' 
final = locationLines.filter(lambda line: True if line['Location'].lower().find('georgia') >= 0 else False 
                             or True if line['Location'].lower().find('atlanta') >= 0 else False 
                             or True if line['Location'].lower().find(', ga') >= 0 else False)
print("The number of residents in GA are", final.count())


#Step 3 - Displaying the names
names = final.map(lambda line:line['DisplayName'])
names.collect()


('The number of residents in GA are', 185)


['Tony Boyles',
 'pkerl',
 'Nick Larsen',
 'gfritz',
 'Aleksandr Blekh',
 'Michael',
 'Ayush',
 'azoorob',
 'ontek',
 'Aravind R. Yarram',
 'ilya',
 'vkb',
 'Daisuke Aramaki',
 'Alexander Stocko',
 'tempusfugit',
 'Henry Crutcher',
 'Goddard',
 'Matt Simpson',
 'Peter Woolfitt',
 'Giorgi Gzirishvili',
 'matt biskup',
 'Jason W',
 'Peter Mourfield',
 'Magsol',
 'Bob Baxley',
 'Alex Azazel',
 'badjr',
 'mplunney',
 'YC Hu',
 'ryan',
 'Patrick Gerbes',
 'Ilya Lapitan',
 'Dan Anton',
 'pradyumnad',
 'Manish Ranjan',
 'Dato Janez',
 'Psidom',
 'Teresa Madsen',
 'Neuromeda',
 'Brandon',
 'jpm',
 'bradden_gross',
 'Mr. Rooter of Savannah',
 'Mr. Rooter of Southeast GA',
 'Donovan Lacey',
 'Abesalomi Gogatishvili',
 'Khiem Ha',
 'Jenna Kwon',
 'Tedo G.',
 'mlibre',
 'Ahmet Cecen',
 'Jeff',
 'Guy Gordon',
 'C3Theo',
 'niru dyogi',
 'Vinitha Palani',
 'Mac18',
 'Andrew',
 'Aditya Gogoi',
 'gh4x',
 'turtlemonvh',
 'Lewis Rodgers',
 'Tarun Luthra',
 'Jesse Scherer',
 'Devendra Lattu',
 'cosmosa',


### Question-2) Using the Users.xml file, provide the count for all users which joined (CreationDate) in 2017.
### Answer-2) -->

In [6]:
#Step 1 - Filtering out UsersRDD for 'CreationDate' key only, CreationDateLines is the filtered RDD
CreationDateLines= UsersRDD.filter(lambda line: "CreationDate" in line)

#Step 2 - Finding out the creation date for the function
ans = CreationDateLines.filter(lambda line: True if line['CreationDate'].find('2017') >= 0 else False )

print (ans.count())

14239


### Question-3) Using the PostHistory file, count the number of Posts that feature the words “Spark” and “Scala”. Output this to the screen. 
### Answer-3) -->

In [8]:

#Step 1 - Filtering out PostHistoryRDD for 'Text' key only, TextLines is the filtered RDD
TextLines= PostHistoryRDD.filter(lambda line: "Text" in line)

#print (TextLines.take(1))

#Step 2 - Counting the Lines containing  both Spark and Scala word
ans = TextLines.filter(lambda line: (True if line['Text'].lower().find('scala') >= 0 else False) 
                     and (True if line['Text'].lower().find('spark') >= 0 else False))

print('The total number of Scala and Spark words used are:',ans.count())


('The total number of Scala and Spark words used are:', 211)


### Question-4) - Using the PostHistory file, provide a total count of the words used by each distinct user. In other words, count all words in all posts for each user and display this to screen. You can only identify users by the UserID. You get 15 bonus points if you get the actual DisplayName of the user.
### Answer -->

In [11]:
#print (PostHistoryRDD.take(1))

#Step 1 - Filtering the lines without UserId and Text
UserIdLines = PostHistoryRDD.filter(lambda line: "UserId" in line)
TextLines = UserIdLines.filter(lambda line: "Text" in line)

#Step 2 - Mapping of the UserId and Text
UserId = TextLines.map(lambda line:line['UserId'])

UserId = TextLines.map(lambda line: (line['UserId'],line['Text']))

#Step 3 - Counting the number of words
ID_WordCount = UserId.map(lambda line:(line[0],len(line[1].split(" "))))

WordCount = ID_WordCount.reduceByKey(lambda x, y: x + y)

#####Combining display name
UserName = UsersRDD.map(lambda line: (line['Id'],line['DisplayName']))

###Step 4 - Combining the final answer with the Display Name
final = WordCount.join(UserName)
print(final.collect())

[('35544', (416, 'vikbehal')), ('73393', (1363, 'akhetos')), ('73397', (1037, 'codiearcher')), ('63767', (482, 'Ammar Ahmed')), ('964', (3335, 'bogatron')), ('43879', (342, 'Alexey')), ('43875', (188, 'Abhishek Soni')), ('73465', (126, 'CoderEnthusiast')), ('74019', (139, 'GoBackess')), ('60230', (45, 'FABIO ZANINI')), ('75944', (65, 'Galuoises')), ('57290', (75, 'bison72')), ('57298', (140, 'Jaskaran Singh Puri')), ('12381', (247, 'ahjohnston25')), ('278', (92, 'phyrox')), ('79081', (225, 'Yoo Inhyeok')), ('52460', (256, 'j Rodr')), ('46700', (122, 'johnnyheineken')), ('63024', (38, 'Sukhrat Arziyev')), ('63020', (114, 'JohanR')), ('78756', (2197, 'ANIKET SAXENA')), ('24883', (6690, 'Aaditya ura')), ('24887', (256, 'username123')), ('24818', (1593, 'Itachi')), ('60919', (135, 'Abe Kassem')), ('69174', (3062, 'm0nzderr')), ('46869', (119, 'Quan Duong')), ('42832', (582, 'EugVal')), ('42836', (64, 'dino ')), ('38503', (192, 'guy')), ('3513', (159, 'Yerk')), ('68177', (142, 'DanielG')), 

### Question -5) GRADUATE STUDENTS: Using the users.xml, comments.xml and PostHistory.xml files, produce a single file that includes the following information: DisplayName, Number of Comments, total Score and Number of posts. This file should have the users (DisplayName) sorted by score, descending from higher to lower. 
### Answer -->

In [213]:
#The 3 RDD files are CommentsRDD , PostHistoryRDD , UsersRDD

###Step 1 - Extracting only the neccesary coloumns from the RDDs
ID_DisplayName = UsersRDD.filter(lambda line: 'Id' in line).map(lambda line:(line['Id'],line['DisplayName']))
#print (ID_DisplayName.collect())

Comments = CommentsRDD.filter(lambda line: 'UserId' in line).map(lambda line: (line['UserId'],int(line['Score'])))
#print(Comments.collect())

CommentsR = CommentsRDD.filter(lambda line: 'UserId' in line).map(lambda line:line['UserId']).map(lambda a: (a, 1))
#print(CommentsR.collect())

Posts = PostHistoryRDD.filter(lambda line: 'UserId' in line).map(lambda line: (line['UserId']))
PostsR = Posts.map(lambda a: (a, 1))

###Step2 - Summarization of Score,post,comments

#Score addition
T_Score = Comments.reduceByKey(lambda x, y: x + y)
#print(T_Score.collect())

#Comment Addition
T_Comments = CommentsR.reduceByKey(lambda x, y: x + y)
#print(T_Comments.collect())

#TotalPost
T_Posts = PostsR.reduceByKey(lambda x, y: x + y)
#print(T_Posts.collect())


### Step 3 - Combination of the above tables
ans1 = T_Score.join(T_Comments)
ans2 = ans1.join(T_Posts)
Final = ans2.join(ID_DisplayName)
#print(Final.collect()) --> This is the RDD which consists of UserID,TotalScore,TotalComments,TotalPost,DisplayName

GeneratedRDD = Final.map(lambda line:(line[1][0][0][0],line[1][1],line[1][0][0][1],line[1][0][1]))

#The required RDD is --> DisplayName,TotalComments,TotalScore,TotalPost, the following is the converstion
FinalRDD = Final.map(lambda line:(line[1][1],line[1][0][0][1],line[1][0][0][0],line[1][0][1]))
print(FinalRDD.collect())



[('vikbehal', 7, 2, 6), ('akhetos', 9, 0, 29), ('codiearcher', 1, 0, 13), ('Ammar Ahmed', 3, 0, 10), ('Jeevansai Jinne', 3, 0, 4), ('Srividya krishnan', 3, 0, 6), ('CoderEnthusiast', 2, 0, 3), ('Galuoises', 1, 0, 3), ('SAR', 2, 0, 3), ('Jaskaran Singh Puri', 1, 0, 6), ('phyrox', 1, 1, 2), ('Nougat Xu', 1, 0, 4), ('LaSul', 6, 0, 5), ('ANIKET SAXENA', 3, 0, 18), ('Aaditya ura', 1, 0, 26), ('username123', 2, 0, 4), ('MrMulliner', 1, 0, 4), ('Abe Kassem', 1, 0, 3), ('m0nzderr', 1, 0, 16), ('Quan Duong', 1, 0, 3), ('Daniel', 9, 0, 15), ('guy', 4, 0, 6), ('Abhinav Thapper', 1, 0, 4), ('oppia04', 2, 0, 3), ('Kechen', 3, 0, 3), ('Lucien Ledune', 1, 0, 6), ('EdChum', 5, 1, 10), ('MD Luffy', 1, 0, 9), ('Leo Brueggeman', 2, 0, 5), ('craq', 3, 0, 5), ('FrankZhu', 2, 0, 1), ('Bador Uddin', 1, 0, 3), ('thovex', 1, 0, 3), ('Mohamed Bakr', 1, 0, 3), ('Tankske', 2, 0, 3), ('justhalf', 1, 0, 3), ('scrlk', 1, 0, 3), ('SriA', 1, 0, 4), ('Dirk Eddelbuettel', 9, 26, 3), ('4Oh4', 1, 1, 3), ('Karthikeyan Naga